<a href="https://colab.research.google.com/github/nutapol97/Python-for-DS-AI_Nutapol_T./blob/main/Classification_Gradient_Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy.special import expit
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
import numpy as np
import matplotlib.pyplot as plt
import xgboost
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import load_digits
from sklearn.ensemble import GradientBoostingRegressor
def train_test(X,Y,size):
  from numpy.random import default_rng
  import math
  rng = default_rng()
  SizeTrain=math.ceil((size*X.shape[0]))
  random_np=rng.choice(X.shape[0], size=SizeTrain, replace=False)
  X_test=X[random_np,:]
  y_test=Y[random_np,]
  Y_train=np.delete(Y, random_np,axis=0)
  X_train=np.delete(X, random_np,axis=0)
  Y_train=Y_train.reshape(Y_train.shape[0],)
  y_test=y_test.reshape(y_test.shape[0],)
  return X_train,X_test,Y_train,y_test
class GradientBoosting:
    def __init__(self, S=5, alfa=1, max_depth = 1, 
                 min_samples_split = 2,
                 regression=True, tol=1e-4):
        self.S = S
        self.alfa = alfa
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.regression=regression

    def grad(self, y, h):
        return y - h
    def predict(self, X, models=None, with_argmax=True):
        if models is None:
            models = self.models
        h0 = models[0].predict(X) 
        boosting = sum(self.alfa * model.predict(X) for model in models[1:])
        yhat = h0 + boosting
        if not self.regression:
            yhat = np.exp(yhat) / np.sum(np.exp(yhat), axis=1, keepdims=True)
            if with_argmax:
                yhat = np.argmax(yhat, axis=1)
        return yhat
    def fit(self, X, y):  
        tree_params = {'max_depth': self.max_depth,
                      'min_samples_split': self.min_samples_split}
        self.models = [DecisionTreeRegressor(**tree_params) for _ in range(self.S)]        
        first_model = DummyRegressor(strategy='mean')
        self.models.insert(0, first_model)
        self.models[0].fit(X, y)
        
        for i in range(self.S):
            yhat = self.predict(X, self.models[:i+1], with_argmax=False)
            gradient = self.grad(y, yhat)
            self.models[i+1].fit(X, gradient)
    
    

In [2]:
# Regression
X_boston, y_boston = load_boston(return_X_y=True)
X_train_boston, X_test_boston, y_train_boston, y_test_boston = train_test(X_boston, y_boston, 
                        size=0.3)

model = GradientBoosting(S=200, alfa=0.1, max_depth = 3, 
                 min_samples_split = 2,
                 regression=True, tol=1e-4)
model.fit(X_train_boston, y_train_boston)
yhat = model.predict(X_test_boston)
print("MSE: ", mean_squared_error(y_test_boston, yhat))
sklearn_model = GradientBoostingRegressor(n_estimators=200,learning_rate = 0.1,max_depth=3,loss='ls')
yhat_sk = sklearn_model.fit(X_train_boston, y_train_boston).predict(X_test_boston)
print("Sklearn MSE: ", mean_squared_error(y_test_boston, yhat_sk))

MSE:  11.343807038149848
Sklearn MSE:  11.240423611089325


In [3]:
# Binary classification
X_cancer, y_cancer = load_breast_cancer(return_X_y=True)
X_train_cancer, X_test_cancer, y_train_cancer, y_test_cancer = train_test(X_cancer, y_cancer, size=0.3)
y_train_cancer_encoded = np.zeros((y_train_cancer.shape[0], len(set(y_cancer))))
for each_class in range(len(set(y_cancer))):
    cond = y_train_cancer==each_class
    y_train_cancer_encoded[np.where(cond), each_class] = 1

model = GradientBoosting(S=200, alfa=0.1, max_depth = 3, min_samples_split = 2,regression=False)
model.fit(X_train_cancer, y_train_cancer_encoded)
yhat_cancer = model.predict(X_test_cancer)

print("Our accuracy: ", accuracy_score(y_test_cancer, yhat_cancer))

sklearn_model = GradientBoostingClassifier(n_estimators=200,learning_rate = 0.1,max_depth=1)
yhat_sk = sklearn_model.fit(X_train_cancer, y_train_cancer).predict(X_test_cancer)
print("Sklearn accuracy: ", accuracy_score(y_test_cancer, yhat_sk))

Our accuracy:  0.9298245614035088
Sklearn accuracy:  0.9532163742690059


In [4]:
# Multiclass classification
X_digits, y_digits = load_digits(return_X_y=True)
X_train_digits, X_test_digits, y_train_digits, y_test_digits = train_test(X_digits, y_digits, size=0.3)
y_train_digits_encoded = np.zeros((y_train_digits.shape[0], len(set(y_digits))))
for each_class in range(len(set(y_digits))):
    cond = y_train_digits==each_class
    y_train_digits_encoded[np.where(cond), each_class] = 1
model = GradientBoosting(S=200, alfa=0.1, max_depth = 3,min_samples_split = 2,regression=False)
model.fit(X_train_digits, y_train_digits_encoded)
yhat = model.predict(X_test_digits)
print("Our accuracy: ", accuracy_score(y_test_digits, yhat))
sklearn_model = GradientBoostingClassifier(n_estimators=200,learning_rate = 0.1,max_depth=1)
yhat_sk = sklearn_model.fit(X_train_digits, y_train_digits).predict(X_test_digits)
print("Sklearn accuracy: ", accuracy_score(y_test_digits, yhat_sk))

Our accuracy:  0.9296296296296296
Sklearn accuracy:  0.9537037037037037
